In [ ]:
# modules and functions
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import random
import pandas as pd
import os
from sklearn.mixture import BayesianGaussianMixture
from pysam import FastaFile
import random
import scipy
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import umap
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfTransformer
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support
from numba import cuda 
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl

# set tick width
mpl.rcParams['xtick.major.size'] = 20
mpl.rcParams['xtick.major.width'] = 4
mpl.rcParams['xtick.minor.size'] = 10
mpl.rcParams['xtick.minor.width'] = 2

mpl.rcParams['ytick.major.size'] = 20
mpl.rcParams['ytick.major.width'] = 4
mpl.rcParams['ytick.minor.size'] = 10
mpl.rcParams['ytick.minor.width'] = 2
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2 
from sklearn import svm 
from sklearn.preprocessing import MinMaxScaler,StandardScaler,MaxAbsScaler,RobustScaler
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, make_scorer,confusion_matrix
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingClassifier
filenames=[]
for i in os.listdir("Data/"):
    if len(i.strip('.csv'))==7:
        filenames.append("Data/"+i)
filenames
x=pd.read_csv("Data/SRR8788980.csv")
x
x.columns
y=np.array(x["DANG_PANCREAS"])
x=x.drop(["Unnamed: 0"],axis=1)
y=np.array(y).reshape(16109   ,)
y_1=np.where(y<=0.5)[0]
y_2=np.where(y>=0.5)[0]
len(y_1),len(y_2)
index=np.random.choice(y_1,5025, replace=False)
index=np.concatenate((index, y_2), axis=0)
index
def classification_report_with_accuracy_score(y_true, y_pred):
    originalclass.extend(y_true)
    predictedclass.extend(y_pred)
    return accuracy_score(y_true, y_pred) # return accuracy score
def confidence_score(y_true,y_pred_prob):

    keys=[0,1]
    dict={}
    i=0
    for key in keys:
        dict[key]=i
        i+=1

    cs=0.0
    for i in range(len(y_true)):
        cs+=y_pred_prob[i][dict[y_true[i]]]


    return cs/len(y_true)
x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
x_32
y_32=y
def normalize(x):
    new_x=[]
    
    for i in range(len(x)):
        x_new=[]
        for j in range(16,32):
            a=np.sum(x[i][0:16])
            b=np.sum(x[i][16:32])
            if x[i][j-16]!=0:
                x_new.append(x[i][j]*a/( x[i][j-16]*b))
            else:
                x_new.append(0.0)
        new_x.append(x_new)
    return new_x
x_32.shape
x_32_norm=np.array(normalize(np.array(x_32)))
x_32_norm.shape
title=['AA', 'AT', 'AG', 'AC',
       'TA', 'TT', 'TG', 'TC', 'GA', 'GT', 'GG', 'GC', 'CA',
       'CT', 'CG', 'CC']
np.sort(title)
x.iloc[np.where(x_32_norm[:,3]>100)][["AA_x","AA_y"]]
plt.subplots(4,4,figsize=(20,20))
for i in range(16):
    plt.title(title[i])
    plt.subplot(4,4,i+1)
    CG=x_32_norm[:,i]
    print(title[i],": ",np.mean(CG[y==0]),np.mean(CG[y==1]),100*(np.mean(CG[y==1])-np.mean(CG[y==0]))/np.mean(CG[y==0]),"%")
    plt.violinplot((CG[y==0],CG[y==1]))
plt.show()
x_32_norm,y
# drop y with neagtive values remove also the corresponding x
index=np.where(y>=0)[0]
x_32_norm=x_32_norm[index]
y=y[index]
# ML models
# Naive Bayes
clf=MultinomialNB(fit_prior=True, class_prior=None)  
clf_parameters = {
        'clf__alpha':(0.1,0.5,1,10,0.01,100,0.001),
        }
pipeline = Pipeline([
    ('clf', clf)]) 


parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=cv)          
grid.fit(x_32_norm,y)     
clf_nb=grid.best_estimator_
clf_nb
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_nb, X=x_32_norm, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
x_norm =StandardScaler().fit_transform(x_32_norm)
# Logistic regression
clf=LogisticRegression(class_weight='balanced',max_iter=1000) 
clf_parameters = {
        'clf__solver':('newton-cg','lbfgs','liblinear'),
        'clf__C':[1,0.1,0.01]
        }

pipeline = Pipeline([   
    ('clf', clf),]) 
parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_macro',cv=cv)          
grid.fit(x_norm,y)     
clf_lr= grid.best_estimator_  
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_lr, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
clf_lr.steps[0][1]
importance = clf_lr.steps[0][1].coef_
plt.bar(title , [ np.linalg.norm(importance[:,i])  for i in range(len(importance[0]))] )
plt.xlabel("Model Parameters")
plt.ylabel("Feature Importance")
plt.show()
y_predict_prob=clf_lr.steps[0][1].predict_proba(x_norm)
confidence_score(y,y_predict_prob)
# Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=40)
clf_parameters = {
        'clf__criterion':('gini', 'entropy'), 
        'clf__max_features':( 'sqrt', 'log2'),
        'clf__ccp_alpha':(0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1),
        'clf__max_depth':(10,40,45,60),
        }
pipeline = Pipeline([
    ('clf', clf)]) 
parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_macro',cv=cv)          
grid.fit(x_norm,y)     
clf_dtc = grid.best_estimator_ 
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_dtc , X=x_norm, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
importance = clf_dtc.steps[0][1].feature_importances_
plt.bar(title, importance)
plt.xlabel("Model Parameters")
plt.ylabel("Feature Importance")
plt.show()
y_predict_prob=clf_dtc.steps[0][1].predict_proba(x_norm)
confidence_score(y,y_predict_prob)
# SVC
clf = svm.SVC(class_weight='balanced',max_iter=5000,probability=True)  
clf_parameters = {
        'clf__C':(0.1,1,2,10,50,100),
        "clf__kernel":('linear','rbf','poly','sigmoid')
        }
pipeline = Pipeline([
    ('clf', clf)]) 

parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
print(cv) 
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=cv)          
grid.fit(x_norm,y)     
clf_svc= grid.best_estimator_ 
clf_svc.steps[0][1].decision_function_shape
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_svc, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
# Deep Learning
clf_dl = MLPClassifier(hidden_layer_sizes=(100,50,10,1),activation='logistic',solver="adam",batch_size=10,learning_rate="adaptive",max_iter=1000)
clf_parameters = {
                    'clf__activation':(['logistic']) #'tanh', 'relu',
                    }

pipeline = Pipeline([
    ('clf', clf)]) 

parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=cv)          
grid.fit(x,y)     
clf= grid.best_estimator_ 
clf_dl
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_dl, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
print(classification_report(originalclass, predictedclass))
# clf_dl.fit(x,y)
plt.plot(clf.steps[0][1].loss_curve_)
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Training Loss")
print(classification_report(originalclass, predictedclass))
def classifier(x):
    classify = tf.keras.Sequential([ layers.Dense(1,activation='softmax',use_bias=False  )])
    output = classify(x)
        
    model = Model(inputs=x, outputs=output)

    return model
model = classifier(layers.Input(shape=(16,)) )
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07 
    ) , loss=  'categorical_crossentropy' ,metrics = 'accuracy')
history = model.fit( x_norm,y   , batch_size=50,
                epochs=10, 
                )
model.weights
# Random forest classifier
clf = RandomForestClassifier(class_weight='balanced')
clf_parameters = {
                    'clf__criterion':('gini', 'entropy'), 
                    'clf__max_features':( 'sqrt', 'log2'),   
                    'clf__n_estimators':(30,50,100,200,400,800,1000,1200),
                    'clf__max_depth':(2,5,10,15,20,25,30),
                    }
pipeline = Pipeline([
    ('clf', clf)]) 

parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=cv)          
grid.fit(x_norm,y)     
clf_rf= grid.best_estimator_ 
clf_rf
grid.best_estimator_ 
clf_rf.steps[0][1]
clf
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_rf, X=x, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))
outer_cv
print(classification_report(originalclass, predictedclass))
importance = clf_rf.steps[0][1].feature_importances_
plt.bar(title, importance)
plt.xlabel("Model Parameters")
plt.ylabel("Feature Importance")
plt.show()
# Use on other data
model = LogisticRegression() #RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=400)
model.fit(x_norm,y)
y
model.predict_proba(x_norm)[:,1]
model.predict(x_norm)
def data_extraction_for_ML(x):
    x_32=x[[ 'AA_x', 'AT_x', 'AG_x', 'AC_x',
       'TA_x', 'TT_x', 'TG_x', 'TC_x', 'GA_x', 'GT_x', 'GG_x', 'GC_x', 'CA_x',
       'CT_x', 'CG_x', 'CC_x', 'AA_y', 'AT_y', 'AG_y', 'AC_y', 'TA_y', 'TT_y',
       'TG_y', 'TC_y', 'GA_y', 'GT_y', 'GG_y', 'GC_y', 'CA_y', 'CT_y', 'CG_y',
       'CC_y']]
    x_32_norm=np.array(normalize(np.array(x_32)))
    
    x_norm =StandardScaler().fit_transform(x_32_norm)
   
    return x_norm
xc.shape
#accuracy_score(y,model.predict(xc))
xtfull=data_extraction_for_ML( pd.read_csv('/project/ReadStatistics-data/TAMIL_FINALFILE.csv',low_memory=False))
xcfull=data_extraction_for_ML( pd.read_csv('/project/ReadStatistics-data/CEPH_FINALFILE.csv',low_memory=False))
xjfull=data_extraction_for_ML( pd.read_csv('/project/ReadStatistics-data/JAPAN_FINALFILE.csv',low_memory=False))
plt.figure(figsize=(30,10))
plt.scatter(np.arange(50),model.predict(xtfull)[0:50],label='TAMIL')
plt.scatter(np.arange(50),model.predict(xcfull)[0:50],label='CEPH')
plt.scatter(np.arange(50),model.predict(xjfull)[0:50],label='JAPAN')
plt.legend()
plt.show()
data = pd.read_csv('/project/ReadStatistics-data/CEPH_FINALFILE.csv',low_memory=False)
merged_data = pd.merge(data, pd.read_csv("/project/ReadStatistics-data/hg38_methylation_level.csv")[['chr','start','end','methy']] , on=['chr','start','end'])
merged_data
xt=data_extraction_for_ML(merged_data)
merged_data
binary_output = np.array(merged_data['methy'])
binary_output[binary_output<0.5]=0
binary_output[binary_output>0.5]=1
model.predict(xt).astype(int)
np.array(binary_output)
accuracy_score(model.predict(xt).astype(np.int32) , np.array(binary_output).astype(np.int32) )
accuracy_score(model.predict(xt).astype(np.int32) , np.array(binary_output).astype(np.int32) )
plt.hist(binary_output)
plt.hist(model.predict(xt))
plt.show()
plt.hist(merged_data['categorize'])
plt.hist(model.predict(xt))

plt.show()
plt.plot(model.predict(xjfull)[0:500])
plt.plot(model.predict(xtfull)[0:500])
plt.plot(model.predict(xcfull)[0:500])

# Gradient boosting classifier
clf = GradientBoostingClassifier()
clf_parameters = {
                    'clf__learning_rate':(0.01,0.1,1,10), 
                    'clf__max_features':( 'sqrt', 'log2'),   
                    'clf__n_estimators':(30,50,100,200,500,800,1000,1200,1400),
                    'clf__max_depth':(2,5,10,15,20,25,30),
                    }
pipeline = Pipeline([
    ('clf', clf)]) 

parameters={**clf_parameters} 
cv = KFold(n_splits=10, shuffle=True, random_state=1)
grid = GridSearchCV(pipeline,parameters,scoring='f1_micro',cv=cv)          
grid.fit(x_norm,y)     
clf_gb= grid.best_estimator_ 
clf_gb
clf.steps[0][1]
len(x) , len(y)
originalclass = []
predictedclass = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

nested_score = cross_val_score(clf_gb, X=x_norm, y=y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

print(classification_report(originalclass, predictedclass))
print(classification_report(originalclass, predictedclass))
importance = clf_gb.steps[0][1].feature_importances_
plt.bar(title, importance)
plt.xlabel("Model Parameters")
plt.ylabel("Feature Importance")
plt.show()
# Models
models = [clf_nb,clf_lr,clf_dtc,clf_svc,clf_dl,clf_rf,clf_gb]
clf_gb
# roc curves
x_32_norm,y
from sklearn.metrics import roc_curve, roc_auc_score
import time

originalclass_models = []
predictedprob_models = []

time_benchmark=[]
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
originalclass=[]
predictedprob=[]

print(models[0])

start=time.time()

for train_index,test_index in outer_cv.split(x,y):
    x_train=x[train_index]
    y_train=y[train_index]

    x_test=x[test_index]
    y_test=y[test_index]

    models[0].fit(x_train,y_train)

    originalclass.extend(y_test)
    predictedprob.extend(models[0].predict_proba(x_test))
    
end=time.time()

time_benchmark.append(end-start)

originalclass_models.append(originalclass)
predictedprob_models.append(predictedprob)
labels_space=[i.replace(" ", "\n") for i in labels]
time_benchmark
# time for training
plt.figure(figsize=(13,7))
plt.scatter(labels_space,np.array(time_benchmark)/10,s=100)
plt.xlabel("Models",fontsize=30)
plt.ylabel("Time (seconds)",fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.tight_layout()

plt.savefig("TIME.pdf",dpi=500)


plt.show()
def binary(x):
    x=np.array(x)
    x[x>0.5]=1.0
    x[x<0.5]=0.0
    return x
labels
for i in range(7):
    print(classification_report(originalclass_models[i],np.argmax(predictedprob_models[i],axis=1) ))
models = [ Pipeline(steps=[('clf', MultinomialNB(alpha=10))]),
Pipeline(steps=[('clf',
                 LogisticRegression(C=0.01, class_weight='balanced',
                                    max_iter=1000, solver='liblinear'))]),
Pipeline(steps=[('clf',
                 DecisionTreeClassifier(ccp_alpha=0.01, max_depth=40,
                                        max_features='sqrt',
                                        random_state=40))]),
Pipeline(steps=[('clf',
                 SVC(C=2, class_weight='balanced', max_iter=5000,
                     probability=True))]),
MLPClassifier(activation='logistic', batch_size=10,
              hidden_layer_sizes=(100, 50, 10, 1), learning_rate='adaptive',
              max_iter=1000),
Pipeline(steps=[('clf',
                 RandomForestClassifier(class_weight='balanced', max_depth=20,
                                        n_estimators=400))]),
Pipeline(steps=[('clf',
                 GradientBoostingClassifier(learning_rate=0.01, max_depth=20,
                                            max_features='log2',
                                            n_estimators=1000))]) ]
for i in models:
    print(i)
x = x_32_norm
for i in range(1,7):

    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    originalclass=[]
    predictedprob=[]
    
    print(models[i])
    
    model=models[i]
    
    start=time.time()


    for train_index,test_index in outer_cv.split(x_norm,y):
        x_train=x[train_index]
        y_train=y[train_index]

        x_test=x[test_index]
        y_test=y[test_index]

        model.fit(x_train,y_train)

        originalclass.extend(y_test)
        predictedprob.extend(model.predict_proba(x_test))
        
    end=time.time()

    time_benchmark.append(end-start)
    
    originalclass_models.append(originalclass)
    predictedprob_models.append(predictedprob)
    
    print("END")
# binarize y
y[y>0.5]=1
y[y<0.5]=0
import os
import joblib
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold

# Create checkpoint directory if not exists
checkpoint_dir = "checkpoint_singlesample"
os.makedirs(checkpoint_dir, exist_ok=True)

for i in range(1, 7):
    outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
    originalclass = []
    predictedprob = []
    
    print(f"Training model: {models[i]}")

    model = models[i]
    start = time.time()

    for fold, (train_index, test_index) in enumerate(outer_cv.split(x, y)):
        x_train, y_train = x[train_index], y[train_index]
        x_test, y_test = x[test_index], y[test_index]

        model.fit(x_train, y_train)

        originalclass.extend(y_test)
        predictedprob.extend(model.predict_proba(x_test))

        # Save model at each fold
        joblib.dump(model, f"{checkpoint_dir}/model_{i}_fold_{fold}.joblib")

    end = time.time()
    time_benchmark.append(end - start)

    originalclass_models.append(originalclass)
    predictedprob_models.append(predictedprob)

    # Save checkpoint results after training each model
    np.savez(f"{checkpoint_dir}/checkpoint_model_{i}.npz", 
             originalclass=np.array(originalclass, dtype=object), 
             predictedprob=np.array(predictedprob, dtype=object), 
             time_taken=end - start)

    print("Model", i, "training complete and checkpoint saved.")
import glob
import joblib


models_restored = []
for i in range(1, 7):  # Model indices
    checkpoint_files = sorted(glob.glob(f"Checkpoints/model_{i}_fold_*.joblib"))
    if checkpoint_files:
        latest_checkpoint = checkpoint_files[-1]  # Get the latest fold
        models_restored.append(joblib.load(latest_checkpoint))
        print(f"Loaded Model {i} from {latest_checkpoint}")
    else:
        models_restored.append(None)  # Placeholder if no model found
len(x_32_norm), len(y)
x_32_norm[0]
models
originalclass_models = []
predictedprob_models = []
time_benchmark = []

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for idx, model in enumerate(models_restored):
    print(f"Evaluating model {idx+1}")

    originalclass = []
    predictedprob = []

    start = time.time()

    j = 0
    for train_index, test_index in outer_cv.split(x_32_norm, y):
        model_dir = f"Checkpoints/model_{idx+1}_fold_{j}.joblib"
        model = joblib.load(model_dir)
        j += 1
        # load model for the fold and checkpoint
        print(train_index)
        x_test = x_32_norm[test_index]
        y_test = y[test_index]

        originalclass.extend(y_test)
        predictedprob.extend(model.predict_proba(x_test))  


    end = time.time()
    time_benchmark.append(end - start)

    originalclass_models.append(originalclass)
    predictedprob_models.append(predictedprob)

print("Evaluation Complete ✅")
len(models)
labels=["Logistic Regression","Decision Tree Classifier","SVC","MLP Classifier","Random Forest Classifier","Gradient Boosting Classifier"]
len(originalclass_models)
plt.figure(figsize=(15,10))


for i in range(6):
    
    fpr, tpr, roc_auc = roc_curve(originalclass_models[i],np.array(predictedprob_models[i])[:,1])
    plt.plot(fpr, tpr,label= labels[i] +" AUC: "+str(round(roc_auc_score(originalclass_models[i],np.array(predictedprob_models[i])[:,1]),2)),linewidth=4.0)
    

plt.plot([0,1],[0,1],label="Random Classifier AUC: 0.50",linestyle = '--',linewidth=4.0,color="black")    
    
plt.legend( prop={'size': 20})
plt.xlabel("False positive rate",fontsize=30)
plt.ylabel("True positive rate",fontsize=30)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.savefig("Figures/ROC.pdf",dpi=500)

plt.show()
np.savez("Figures/roc_data.npz", 
         originalclass_models=np.array(originalclass_models),  
         predictedprob_models=np.array(predictedprob_models),  
         labels=np.array(labels))
originalclass_models
from sklearn.metrics import precision_recall_curve , average_precision_score
# pr curves
plt.figure(figsize=(15,10))
for i in range(6):
    
    precision, recall, thresholds  = precision_recall_curve(originalclass_models[i],np.array(predictedprob_models[i])[:,1])
    plt.plot(recall,precision,label= labels[i] +" AP: "+str(round(average_precision_score(originalclass_models[i],np.array(predictedprob_models[i])[:,1]),2)),linewidth=4.0)
    
plt.legend( prop={'size': 20})
plt.xlabel("Recall",fontsize=30)
plt.ylabel("Precision",fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.savefig("Figures/PR.pdf")

plt.show()
# Feature importances 
from sklearn.inspection import permutation_importance
features=[]
for i in "ATGC":
    for j in "ATGC":
        features.append(i+j)

features_sort=[]
for i in "ACGT":
    for j in "ACGT":
        features_sort.append(i+j)
    
index_sorted=[]
for i in range(16):
    for j in range(16):
        if features[i]==features_sort[j]:
            index_sorted.append(j) 


plt.figure(figsize=(10,6))
j=0
for i in results:
    plt.bar( np.arange(0,32,2)+j*0.1,np.mean(i[3],axis=0)[index_sorted],label=cancer_types[j], width = 0.05)
    j+=1
    
plt.bar( np.arange(0,32,2)+j*0.1,np.mean(results_human_20[0][3],axis=0)[index_sorted],label="HUMAN BLOOD 20 SAMPLES", width = 0.05)

plt.xticks(np.arange(0,32,2)+5*0.1,np.array(features)[index_sorted],fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel("\nFeatures",fontsize=20)
plt.ylabel("\nFeature importances",fontsize=20)
plt.legend(bbox_to_anchor=(1.0, 1.0),title="Sample names",fontsize=15,title_fontsize=15)

plt.tight_layout()


plt.savefig("/project/ReadStatistics/ML_Cancer_multiple/feature_importances.png",dpi=800)

plt.show()
# plot the feature importance for each model

for i in range(5):
    importance = feature_importances[i]
    plt.bar(title, importance, label = labels[i])
plt.legend()
plt.xlabel("Model Parameters")
plt.ylabel("Feature Importance")
plt.title("Feature Importance of Different Models")
plt.show()
